In [1]:
import time
from pynq.overlays.base import BaseOverlay
import socket

base = BaseOverlay("base.bit")
btns = base.btns_gpio
leds = base.leds

# Sockets

This notebook has both a client and a server functionality. One PYNQ board in the group will be the client and SENDS the message. Another PYNQ board will be the server and RECEIVES the message.

## Server

Here, we'll build the server code to LISTEN for a message from a specific PYNQ board.

When we send/receive messages, we need to pieces of information which will tell us where to send the information. First, we need the IP address of our friend. Second, we need to chose a port to listen on. For an analogy, Alice expects her friend, Bob, to deliver a package to our back door. With this information, ALICE (server ip address) can wait at the BACK DOOR (port) for BOB (client ip address) to deliver the package.

Format of the information
 ipv4 address: ###.###.###.### (no need for leading zeros if the number is less than three digits)
 port: ##### (it could be 4 or 5 digits long, but must be >1024)
 
Use the socket documentation (Section 18.1.3) to find the appropriate functions https://python.readthedocs.io/en/latest/library/socket.html


In [9]:
import socket

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Bind the socket to the port
server_address = ('192.168.0.101', 12345)  
sock.bind(server_address)

# Listen for incoming connections
sock.listen()

# Wait for a connection
print('waiting for a connection')
connection, client_address = sock.accept()

try:
    print('connection from', client_address)
    
    # Receive the data in small chunks
    while True:
        data = connection.recv(16)
        print('received {!r}'.format(data))
        if not data:
            break

finally:
    # Clean up the connection
    connection.close()
    sock.close()

OSError: [Errno 98] Address already in use

## Client

Now, we can implement the CLIENT code. 

Back to the analogy, now we're interested in delivering a package to our friend's back door. This means BOB (client ip address) is delivering a package to ALICE (server ip address) at her BACK DOOR (port)

**Remember to start the server before running the client code**

In [10]:
import socket

# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the server
print("Attempting to connect...")
server_address = ('192.168.0.104', 65432)  
sock.connect(server_address)
print("Client connected.")


try:
    # Send data
    message = 'Hello world!\n Hi\n Saturday\n '
    sock.sendall(message.encode())

finally:
    # Clean up the connection
    sock.close()

Attempting to connect...
Client connected.


On your server, you should see the message and then the server will shutdown! When we close a socket, both the client and the server are disconnected from the port.

Instead, change the function above to send 5 messages before closing.

The pseudocode looks like this

* connect the socket
* for i in range(5)
    * msg = input("Message to send: ")
    * send the message (msg)
* close the socket